## Deliverable 2. Create a Customer Travel Destinations Map.

In [71]:
# Dependencies and Setup
from IPython.display import clear_output
import time
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [72]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.drop("Unnamed: 0", axis=1, inplace=True)
city_data_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Weather_Description
0,upernavik,72.79,-56.15,20.95,92,100,7.56,GL,2020-10-24 08:17:49,overcast clouds
1,bambous virieux,-20.34,57.76,80.60,54,75,13.87,MU,2020-10-24 08:17:49,broken clouds
2,state college,40.79,-77.86,61.00,87,75,5.82,US,2020-10-24 08:14:25,moderate rain
3,castro,-24.79,-50.01,57.76,95,100,5.14,BR,2020-10-24 08:16:04,overcast clouds
4,rikitea,-23.12,-134.97,74.07,83,100,14.29,PF,2020-10-24 08:17:50,light rain


In [73]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
def temperature_input():
    try:
        min_temp, max_tep = input("Please enter your prefered minumum temperature followed by your prefered maximum temperature seperated by a ',':\n ").strip().split(",")
    except ValueError:
        clear_output()
        print("\nPlease Enter 2 integers seperated by a ','")
        time.sleep(3)
        clear_output()
        min_temp, max_temp = temperature_input()
    return min_temp, max_tep


min_temp, max_temp = temperature_input()
   

Please enter your prefered minumum temperature followed by your prefered maximum temperature seperated by a ',':
 75,90


In [74]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
Max_Min_Temp_df=city_data_df.loc[(city_data_df.Max_Temp >= int(min_temp)) & (city_data_df.Max_Temp <= int(max_temp))]

In [75]:
# 4a. Determine if there are any empty rows.
Max_Min_Temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447 entries, 1 to 1805
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 447 non-null    object 
 1   Lat                  447 non-null    float64
 2   Lng                  447 non-null    float64
 3   Max_Temp             447 non-null    float64
 4   Humidity             447 non-null    int64  
 5   Cloudiness           447 non-null    int64  
 6   Wind_Speed           447 non-null    float64
 7   Country              446 non-null    object 
 8   Date                 447 non-null    object 
 9   Weather_Description  447 non-null    object 
dtypes: float64(4), int64(2), object(4)
memory usage: 38.4+ KB


In [76]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Max_Min_Temp_df=Max_Min_Temp_df.dropna()

In [77]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Max_Min_Temp_df[["City", "Country", "Max_Temp", "Weather_Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max_Temp,Weather_Description,Lat,Lng,Hotel Name
1,bambous virieux,MU,80.60,broken clouds,-20.34,57.76,
6,butaritari,KI,82.35,broken clouds,3.07,172.79,
8,acarau,BR,78.17,few clouds,-2.89,-40.12,
13,busselton,AU,77.00,clear sky,-33.65,115.33,
14,lorengau,PG,82.78,scattered clouds,-2.02,147.27,
17,victoria,HK,77.00,overcast clouds,22.29,114.16,
18,busselton,AU,77.00,clear sky,-33.65,115.33,
22,muli,MV,82.63,broken clouds,2.92,73.57,
23,jieshi,CN,80.01,overcast clouds,22.81,115.83,
24,east london,ZA,77.00,broken clouds,-33.02,27.91,


In [78]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
counter=0
index_2=0
# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():  
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Add lat and lng to params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    response=requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f'Success {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df)*100))}%')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
        
    except IndexError:
        print(f'\nFaliure {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df.index)*100))}% \n\t\tSkipping...\n')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
        pass
print("-"*30+"\nCompleted\n"+"-"*30)

Success 440 of 446 | 98.65%
Success 441 of 446 | 98.88%
Success 442 of 446 | 99.10%
Success 443 of 446 | 99.33%
Success 444 of 446 | 99.55%
Success 445 of 446 | 99.78%


In [79]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [80]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [81]:

### Heat_Layer_Map ###


# 11a. Add a marker layer for each city to the map. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description and Max Temp</dt><dd>{Weather_Description} \tMax Temp: {Max_Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max_Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [82]:

### Marker_Layer_Map ###


# 11a. Add a marker layer for each city to the map. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description and Max Temp</dt><dd>{Weather_Description} \tMax Temp: {Max_Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max_Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5,map_type="HYBRID")
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))